In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import klib

In [ ]:
#load processed data
df = pd.read_csv('KDDTrain+_cleaned.csv')

In [12]:
X = df.drop(['type'], axis=1)
y = df['type']
# one hot encoding
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [13]:
# train an lstm model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='lstm_best_model.h5', verbose=1, save_best_only=True, monitor = 'val_accuracy', mode = 'max')
start = time.time()
# train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpointer])
end = time.time()
print(end - start)

Epoch 1/10
4209/4209 [==============================] - ETA: 0s - loss: 0.3423 - accuracy: 0.8770
Epoch 1: val_accuracy improved from -inf to 0.92010, saving model to lstm_best_model.h5
4209/4209 [==============================] - 649s 154ms/step - loss: 0.3423 - accuracy: 0.8770 - val_loss: 0.1954 - val_accuracy: 0.9201
Epoch 2/10


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4209/4209 [==============================] - ETA: 0s - loss: 0.2036 - accuracy: 0.9248
Epoch 2: val_accuracy improved from 0.92010 to 0.94538, saving model to lstm_best_model.h5
4209/4209 [==============================] - 672s 160ms/step - loss: 0.2036 - accuracy: 0.9248 - val_loss: 0.1449 - val_accuracy: 0.9454
Epoch 3/10
4209/4209 [==============================] - ETA: 0s - loss: 0.1349 - accuracy: 0.9489
Epoch 3: val_accuracy improved from 0.94538 to 0.95048, saving model to lstm_best_model.h5
4209/4209 [==============================] - 675s 160ms/step - loss: 0.1349 - accuracy: 0.9489 - val_loss: 0.1243 - val_accuracy: 0.9505
Epoch 4/10
4209/4209 [==============================] - ETA: 0s - loss: 0.1297 - accuracy: 0.9510
Epoch 4: val_accuracy improved from 0.95048 to 0.95578, saving model to lstm_best_model.h5
4209/4209 [==============================] - 670s 159ms/step - loss: 0.1297 - accuracy: 0.9510 - val_loss: 0.1166 - val_accuracy: 0.9558
Epoch 5/10
4209/4209 [===========

In [14]:
model = load_model('lstm_best_model.h5')
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

2105/2105 [==============================] - 39s 18ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13340
           1       1.00      1.00      1.00     13507
           2       1.00      0.99      1.00     13499
           3       0.98      0.96      0.97     13629
           4       0.96      0.98      0.97     13368

    accuracy                           0.99     67343
   macro avg       0.99      0.99      0.99     67343
weighted avg       0.99      0.99      0.99     67343

0.9870068158531696


In [15]:
# from confusion matrix calculate accuracy of each class
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
print('dos:', cm[0,0])
print('normal:', cm[1,1])
print('probe:', cm[2,2])
print('r2l:', cm[3,3])
print('u2r:', cm[4,4])

Accuracy of each class:
dos: 0.9973013493253373
normal: 0.998889464721996
probe: 0.9919253278020594
r2l: 0.963607014454472
u2r: 0.9836175942549371
